# Xử lí dữ liệu cho fakenews-detection

In [1]:
import pandas as pd
import numpy as np
import re
import string

Đọc dữ liệu từ file csv với pandas

In [2]:
fake_df = pd.read_csv('dataset/Fake.csv')
true_df = pd.read_csv('dataset/True.csv')

Xem thử 5 dòng đầu của mỗi tập dữ liệu fake / real

In [3]:
fake_df.head(5)

,title,text,subject,date
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017"
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017"
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017"
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017"
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017"


In [4]:
true_df.head(5)

,title,text,subject,date
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017"
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017"
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017"
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017"
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017"


## Thêm class đánh dấu tin fake / real

In [5]:
true_df['class'] = 1
fake_df['class'] = 0

In [6]:
true_df.shape

(21417, 5)

In [7]:
fake_df.shape

(23481, 5)

Tách riêng 50 dòng cuối của mỗi loại dành để test manual, tổng cộng sẽ có 100 dòng để test.

In [8]:
fake_df_test = fake_df.tail(50)
true_df_test = true_df.tail(50)

In [9]:
fake_df_test.head(10)

,title,text,subject,date,class
23431,"BOILER ROOM – EP #46 – Murder, Witchery, Polit...",Tune in to the Alternate Current Radio Network...,Middle-east,"March 3, 2016",0
23432,Survey: Top Ten Fears of 2015,Truthstream MediaOut of 88 potential horrors ...,Middle-east,"February 29, 2016",0
23433,SHOUT POLL: Should Apple Give FBI Backdoor Acc...,21st Century Wire asks HAVE YOUR SHOUT: Apple...,Middle-east,"February 27, 2016",0
23434,"The Final Control: TPP, TTIP, TISA Global Corp...",21st Century Wire says This is a new geopoliti...,Middle-east,"February 27, 2016",0
23435,Blood Sport: GOP Presidential Race Takes Anoth...,Patrick Henningsen 21st Century WireAs Baron R...,Middle-east,"February 26, 2016",0
23436,HIDDEN ORDER: Was the Death of Justice Scalia ...,Shawn Helton 21st Century WireWith the Unite...,Middle-east,"February 26, 2016",0
23437,The Android Affair: Humanity Outsourced,Randy Johnson 21st Century WireThe future is c...,Middle-east,"February 25, 2016",0
23438,"BOILER ROOM – EP #45 – Horror Hotel, Trump Gat...",Tune in to the Alternate Current Radio Network...,Middle-east,"February 24, 2016",0
23439,Syria Ceasefire Deal: A Cynical Ploy by Washin...,21st Century Wire says This latest diplomatic ...,Middle-east,"February 24, 2016",0
23440,Watch as Trump Gatecrashes Glenn Beck’s Cruz C...,21st Century Wire says One TV personality Tru...,Middle-east,"February 24, 2016",0


In [10]:
true_df_test.head(10)

,title,text,subject,date,class
21367,"As guns fall silent, Benghazi residents return...","BENGHAZI, Libya (Reuters) - Two months after t...",worldnews,"August 24, 2017",1
21368,Venezuela's injured activists struggle to heal,"CARACAS (Reuters) - Jesus Ibarra, a 19-year-ol...",worldnews,"August 24, 2017",1
21369,India's push to broaden use of its biometric d...,(Reuters) - India s Supreme Court ruled on Thu...,worldnews,"August 24, 2017",1
21370,Finnish police release one knife attack suspect,HELSINKI (Reuters) - Finnish police on Thursda...,worldnews,"August 24, 2017",1
21371,"As Syria war tightens, U.S. and Russia militar...","AL UDEID AIR BASE, Qatar (Reuters) - Even as t...",worldnews,"August 24, 2017",1
21372,U.N. calls for pause in air strikes to spare c...,GENEVA (Reuters) - The United Nations called o...,worldnews,"August 24, 2017",1
21373,EU citizens leaving UK pushes down net migrati...,LONDON (Reuters) - Net migration to Britain fe...,worldnews,"August 24, 2017",1
21374,Excessive force won't solve Myanmar's Rohingya...,YANGON (Reuters) - Myanmar should respond to a...,worldnews,"August 24, 2017",1
21375,Britain to study effect of foreign students on...,LONDON (Reuters) - Britain commissioned a repo...,worldnews,"August 24, 2017",1
21376,Turkish nationalist leader says Iraqi Kurdish ...,ANKARA (Reuters) - The head of Turkey s nation...,worldnews,"August 24, 2017",1


Vì đã lấy 10 dòng cuối của mỗi set làm manual testing data, drop 10 dòng cuối đó khỏi data gốc.

In [11]:
true_df.drop(true_df.tail(50).index, inplace = True)

In [12]:
fake_df.drop(fake_df.tail(50).index, inplace = True)

In [13]:
true_df.shape

(21367, 5)

In [14]:
fake_df.shape

(23431, 5)

Merge 2 tập fake và real lại thành 1 tập train.

In [15]:
train_df = pd.concat([true_df, fake_df], axis = 0)

In [16]:
train_df.shape

(44798, 5)

In [17]:
train_df.head(10)

,title,text,subject,date,class
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017",1
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017",1
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017",1
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017",1
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017",1
5,"White House, Congress prepare for talks on spe...","WEST PALM BEACH, Fla./WASHINGTON (Reuters) - T...",politicsNews,"December 29, 2017",1
6,"Trump says Russia probe will be fair, but time...","WEST PALM BEACH, Fla (Reuters) - President Don...",politicsNews,"December 29, 2017",1
7,Factbox: Trump on Twitter (Dec 29) - Approval ...,The following statements were posted to the ve...,politicsNews,"December 29, 2017",1
8,Trump on Twitter (Dec 28) - Global Warming,The following statements were posted to the ve...,politicsNews,"December 29, 2017",1
9,Alabama official to certify Senator-elect Jone...,WASHINGTON (Reuters) - Alabama Secretary of St...,politicsNews,"December 28, 2017",1


Merge luôn 2 tập fake_test và real_test thành 1 tập test.

In [18]:
test_df = pd.concat([fake_df_test, true_df_test], axis = 0)

In [19]:
test_df.shape

(100, 5)

In [20]:
test_df.head(10)

,title,text,subject,date,class
23431,"BOILER ROOM – EP #46 – Murder, Witchery, Polit...",Tune in to the Alternate Current Radio Network...,Middle-east,"March 3, 2016",0
23432,Survey: Top Ten Fears of 2015,Truthstream MediaOut of 88 potential horrors ...,Middle-east,"February 29, 2016",0
23433,SHOUT POLL: Should Apple Give FBI Backdoor Acc...,21st Century Wire asks HAVE YOUR SHOUT: Apple...,Middle-east,"February 27, 2016",0
23434,"The Final Control: TPP, TTIP, TISA Global Corp...",21st Century Wire says This is a new geopoliti...,Middle-east,"February 27, 2016",0
23435,Blood Sport: GOP Presidential Race Takes Anoth...,Patrick Henningsen 21st Century WireAs Baron R...,Middle-east,"February 26, 2016",0
23436,HIDDEN ORDER: Was the Death of Justice Scalia ...,Shawn Helton 21st Century WireWith the Unite...,Middle-east,"February 26, 2016",0
23437,The Android Affair: Humanity Outsourced,Randy Johnson 21st Century WireThe future is c...,Middle-east,"February 25, 2016",0
23438,"BOILER ROOM – EP #45 – Horror Hotel, Trump Gat...",Tune in to the Alternate Current Radio Network...,Middle-east,"February 24, 2016",0
23439,Syria Ceasefire Deal: A Cynical Ploy by Washin...,21st Century Wire says This latest diplomatic ...,Middle-east,"February 24, 2016",0
23440,Watch as Trump Gatecrashes Glenn Beck’s Cruz C...,21st Century Wire says One TV personality Tru...,Middle-east,"February 24, 2016",0


Bỏ bớt một số cột không cần: title, subject và date

In [21]:
train_df = train_df.drop(['title', 'subject', 'date'], axis = 1)

In [22]:
test_df = test_df.drop(['title', 'subject', 'date'], axis = 1)

In [23]:
train_df.shape

(44798, 2)

In [24]:
test_df.shape

(100, 2)

In [25]:
train_df.isnull().sum()

text     0
class    0
dtype: int64

In [26]:
test_df.isnull().sum()

text     0
class    0
dtype: int64

Trộn ngẫu nhiên data trong train set

In [27]:
train_df = train_df.sample(frac = 1, random_state = 42).reset_index(drop = True)

In [28]:
train_df.head()

,text,class
0,Shawn Helton 21st Century WireOn the eve of th...,0
1,WASHINGTON (Reuters) - Democratic presidential...,1
2,Join Patrick every Wednesday at Independent T...,0
3,Coming to a town near you A sign was hung in a...,0
4,NEW YORK (Reuters) - While Donald Trump’s poli...,1


Cũng làm y chang vậy với test set.

In [29]:
test_df = test_df.sample(frac = 1, random_state = 42).reset_index(drop = True)

In [30]:
test_df.head()

,text,class
0,LONDON (Reuters) - Abdul Daoud spilt most of t...,1
1,HELSINKI (Reuters) - Finnish police on Thursda...,1
2,TRIPOLI (Reuters) - Former Libyan Prime Minist...,1
3,21st Century Wire says As 21WIRE reported earl...,0
4,Robert Fantina CounterpunchAlthough the United...,0


Save dataframes vào file csv.

In [31]:
train_df.to_csv('dataset/train.csv')
test_df.to_csv('dataset/test.csv')